## Proof of concept exploration of null mechanism for SPAWN

Goal: Demonstrate how a null mechanism could in principle result in cross structural priming. 

#### Primes

* SC Prime: The teacher thought that/NULL-THAT1 the student appreciated the idea
* RC Prime: The teacher appreciated the idea that/NULL-THAT2 the student expressed

#### Target
The professor announced ____ the exam will be next week.

Measure: ____ filled with that or NULL-THAT1?

In [87]:
import re
import math
import random
from copy import deepcopy
import numpy as np
import pickle
import ccg
import math

math.log(eps)

-16.11809565095832

## Representational assumptions

**Assumption 1** We store overt categories and null categories as separate syntax chunks

In [75]:
overt_chunks = {
    'Det': {
        'left': 'NP',
        'right': 'N',
        'combinator': '/'
    },
    'Noun': {
        'left': 'N',
        'right': '',
        'combinator': ''
    },
    'SC_verb':{
        'left': '(S\\NP)',
        'right': 'S',
        'combinator': ''
    },
    'Tr_verb':{
        'left': '(S\\NP)',
        'right': 'NP',
        'combinator': '/'
    },
    'that1':{
        'left': 'S',
        'right': 'S',
        'combinator': '/'
    },
    'that2':{
        'left': '(N/N)',
        'right': '(S/NP)',
        'combinator': '/'
    }

}

## these don't have to be structures in syntax_chunks. Can be anything. 
null_chunks = {
    'NULL-THAT1': syntax_chunks['that1'],
    'NULL-THAT2': syntax_chunks['that2']
}

**Assumption 2**: Every syntax chunk (overt and null) is associated with a base-level activation which is influenced by how frequently and recently the chunk was retrieved. (This assumption is the same as in the og SPAWN model) 

**Assumption 3**: Each overt chunk has spreading activation to null chunks which indicates how likely is the overt chunk to be followed by a null chunk. (This is like lexical activation, but instead of being tied to specific words, it is broader and is tied to categories). 

## Set up toy activations

* Currently just based on frequency, without any decay. 
* Exposed to 4 sentences: one for each prime type and overt vs. covert THAT.
* Det and Noun occur in all sentences -- so their value is 4
* 

In [101]:

## These numbers are not looking at entire sentence just the first part. 
null_count_bytag = {
    'Det': {
        'NULL-THAT1': 0,
        'NULL-THAT2': 0,
        'not-null': 4
    },

    'Noun': {
        'NULL-THAT1': 0,
        'NULL-THAT2': 0,
        'not-null': 4
    },

    'SC_verb': {
        'NULL-THAT1': 1,
        'NULL-THAT2': 0,
        'not-null': 1
    },
    'Tr_verb': {
        'NULL-THAT1': 0,
        'NULL-THAT2': 1,
        'not-null': 1
    },

    'that1': {
        'NULL-THAT1': 0,
        'NULL-THAT2': 0,
        'not-null': 2
    },

    'that2': {
        'NULL-THAT1': 0,
        'NULL-THAT2': 0,
        'not-null': 2
    },
}


null_count = {
        'NULL-THAT1': 0,
        'NULL-THAT2': 0,
        'not-null': 0 
    }

for null in null_count: 
    for overt in null_count_bytag:
        null_count[null] += null_count_bytag[overt][null]


print(null_count)

{'NULL-THAT1': 1, 'NULL-THAT2': 1, 'not-null': 14}


## Specify null mechanism

In [103]:
def predict_null(word, curr_tag, parse_state, null_base, null_base_bytag, null_chunks, tr_rules, noise_sd):

    found_valid = False
    poss_tags = list(null_base.keys())

    while not found_valid: 
        ## initialize with base-level count
        act_dict = {tag: null_base[tag] for tag in poss_tags}

        ## add in count from current tag
        for tag in act_dict:
            act_dict[tag] += null_base_bytag[curr_tag][tag]

        ## log transform
        eps = 0.0001
        for tag in act_dict:
            act_dict[tag] = math.log(act_dict[tag] + eps)
        
        
        ## add in noise 
        for tag in act_dict:  
            act_dict[tag] += np.random.normal(0, noise_sd)

        null_pred = max(act_dict, key=lambda key:act_dict[key])

        if null_pred == 'not-null':
            found_valid = True
        else:
            null_chunk = null_chunks[null_pred]

            combined = ccg.combine(
                tag = null_chunk,
                parse_state = parse_state,
                tr_rules = tr_rules)

            if combined:
                found_valid = True
            else: ## invalid tag with parse state
                poss_tags.remove(null_pred) 
    
    return null_pred

Other things to implement/ think through: 

* Lexical activation
* Reanalysis w.r.t. null elements + inhibition 

## Basic priming experiment

**Counts of null-prediction for 1000 runs with no priming**

In [104]:
sc_parse_state = {
    'left': 'S',
    'right': 'S',
    'combinator': '/'
}

counts = {key: 0 for key in null_categories}
counts['not-null'] = 0

for i in range(1000):
    null = predict_null(word = 'announced', 
                 curr_tag = 'SC_verb',
                 parse_state = sc_parse_state,
                 null_base = null_count,
                 null_base_bytag = null_count_bytag,
                 null_chunks = null_chunks,
                 tr_rules = {},
                 noise_sd = 1)
    counts[null]+=1

counts          
             

{'NULL-THAT1': 67, 'NULL-THAT2': 0, 'not-null': 933}

**Counts of null-prediction for 1000 runs with one SC prime with null-that**

In [108]:
null_bytag_scprime = deepcopy(null_count_bytag)
null_bytag_scprime['SC_verb']['NULL-THAT1']+=1
null_bytag_scprime['SC_verb']['not-null']+=2

null_scprime = deepcopy(null_act)
null_scprime['NULL-THAT1'] +=1


counts = {key: 0 for key in null_categories}
counts['not-null'] = 0

for i in range(1000):
    null = predict_null(word = 'announced', 
                 curr_tag = 'SC_verb',
                 parse_state = sc_parse_state,
                 null_base = null_scprime,
                 null_base_bytag = null_bytag_scprime,
                 null_chunks = null_chunks,
                 tr_rules = {},
                 noise_sd = 1)
    counts[null]+=1

counts          


{'NULL-THAT1': 334, 'NULL-THAT2': 0, 'not-null': 666}

**Counts of null-prediction for 1000 runs with one RC prime with null-that**

In [109]:
null_bytag_rcprime = deepcopy(null_count_bytag)
null_bytag_rcprime['Tr_verb']['NULL-THAT2']+=1
null_bytag_scprime['Tr_verb']['not-null']+=2

null_rcprime = deepcopy(null_act)
null_rcprime['NULL-THAT2'] += 1

counts = {key: 0 for key in null_categories}
counts['not-null'] = 0

for i in range(1000):
    null = predict_null(word = 'announced', 
                 curr_tag = 'SC_verb',
                 parse_state = sc_parse_state,
                 null_base = null_rcprime,
                 null_base_bytag = null_bytag_scprime,
                 null_chunks = null_categories,
                 tr_rules = {},
                 noise_sd = 1)
    counts[null]+=1

counts      

{'NULL-THAT1': 213, 'NULL-THAT2': 0, 'not-null': 787}